In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Notebook magic

In [1]:
from IPython.core.magic import Magics, magics_class, line_cell_magic
from IPython.core.magic import cell_magic, register_cell_magic, register_line_magic
from IPython.core.magic_arguments import argument, magic_arguments, parse_argstring
import subprocess
import os

In [2]:
@magics_class
class PyboardMagic(Magics):
    @cell_magic
    @magic_arguments()
    @argument('-skip')
    @argument('-unix')
    @argument('-pyboard')
    @argument('-file')
    @argument('-data')
    @argument('-time')
    @argument('-memory')
    def micropython(self, line='', cell=None):
        args = parse_argstring(self.micropython, line)
        if args.skip: # doesn't care about the cell's content
            print('skipped execution')
            return None # do not parse the rest
        if args.unix: # tests the code on the unix port. Note that this works on unix only
            with open('/dev/shm/micropython.py', 'w') as fout:
                fout.write(cell)
            proc = subprocess.Popen(["../../micropython/ports/unix/micropython", "/dev/shm/micropython.py"], 
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            print(proc.stdout.read().decode("utf-8"))
            print(proc.stderr.read().decode("utf-8"))
            return None
        if args.file: # can be used to copy the cell content onto the pyboard's flash
            spaces = "    "
            try:
                with open(args.file, 'w') as fout:
                    fout.write(cell.replace('\t', spaces))
                    printf('written cell to {}'.format(args.file))
            except:
                print('Failed to write to disc!')
            return None # do not parse the rest
        if args.data: # can be used to load data from the pyboard directly into kernel space
            message = pyb.exec(cell)
            if len(message) == 0:
                print('pyboard >>>')
            else:
                print(message.decode('utf-8'))
                # register new variable in user namespace
                self.shell.user_ns[args.data] = string_to_matrix(message.decode("utf-8"))
        
        if args.time: # measures the time of executions
            pyb.exec('import utime')
            message = pyb.exec('t = utime.ticks_us()\n' + cell + '\ndelta = utime.ticks_diff(utime.ticks_us(), t)' + 
                               "\nprint('execution time: {:d} us'.format(delta))")
            print(message.decode('utf-8'))
        
        if args.memory: # prints out memory information 
            message = pyb.exec('from micropython import mem_info\nprint(mem_info())\n')
            print("memory before execution:\n========================\n", message.decode('utf-8'))
            message = pyb.exec(cell)
            print(">>> ", message.decode('utf-8'))
            message = pyb.exec('print(mem_info())')
            print("memory after execution:\n========================\n", message.decode('utf-8'))

        if args.pyboard:
            message = pyb.exec(cell)
            print(message.decode('utf-8'))

ip = get_ipython()
ip.register_magics(PyboardMagic)

## pyboard

In [57]:
import pyboard
pyb = pyboard.Pyboard('/dev/ttyACM0')
pyb.enter_raw_repl()

In [9]:
pyb.exit_raw_repl()
pyb.close()

In [58]:
%%micropython -pyboard 1

import utime
import ulab as np

def timeit(n=1000):
    def wrapper(f, *args, **kwargs):
        func_name = str(f).split(' ')[1]
        def new_func(*args, **kwargs):
            run_times = np.zeros(n, dtype=np.uint16)
            for i in range(n):
                t = utime.ticks_us()
                result = f(*args, **kwargs)
                run_times[i] = utime.ticks_diff(utime.ticks_us(), t)
            print('{}() execution times based on {} cycles'.format(func_name, n, (delta2-delta1)/n))
            print('\tbest: %d us'%np.min(run_times))
            print('\tworst: %d us'%np.max(run_times))
            print('\taverage: %d us'%np.mean(run_times))
            print('\tdeviation: +/-%.3f us'%np.std(run_times))            
            return result
        return new_func
    return wrapper

def timeit(f, *args, **kwargs):
    func_name = str(f).split(' ')[1]
    def new_func(*args, **kwargs):
        t = utime.ticks_us()
        result = f(*args, **kwargs)
        print('execution time: ', utime.ticks_diff(utime.ticks_us(), t), ' us')
        return result
    return new_func

__END_OF_DEFS__

# scipy.linalg

`scipy`'s `linalg` module contains two functions, `solve_triangular`, and `cho_solve`. The functions can be called by prepending them by `scipy.linalg.`.

1. [scipy.linalg.solve_cho](#cho_solve)
2. [scipy.linalg.solve_triangular](#solve_triangular)

## cho_solve

`scipy`: https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.cho_solve.html

Solve the linear equations 


\begin{equation}
\mathbf{A}\cdot\mathbf{x} = \mathbf{b}
\end{equation}

given the Cholesky factorization of $\mathbf{A}$. As opposed to `scipy`, the function simply takes the Cholesky-factorised matrix, $\mathbf{A}$, and $\mathbf{b}$ as inputs.

In [5]:
%%micropython -unix 1

from ulab import numpy as np
from ulab import scipy as spy

A = np.array([[3, 0, 0, 0], [2, 1, 0, 0], [1, 0, 1, 0], [1, 2, 1, 8]])
b = np.array([4, 2, 4, 2])

print(spy.linalg.cho_solve(A, b))

array([-0.01388888888888906, -0.6458333333333331, 2.677083333333333, -0.01041666666666667], dtype=float64)




## solve_triangular

`scipy`: https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve_triangular.html 

Solve the linear equation 

\begin{equation}
\mathbf{a}\cdot\mathbf{x} = \mathbf{b}
\end{equation}

with the assumption that $\mathbf{a}$ is a triangular matrix. The two position arguments are $\mathbf{a}$, and $\mathbf{b}$, and the optional keyword argument is `lower` with a default value of `False`. `lower` determines, whether data are taken from the lower, or upper triangle of $\mathbf{a}$. 

Note that $\mathbf{a}$ itself does not have to be a triangular matrix: if it is not, then the values are simply taken to be 0 in the upper or lower triangle, as dictated by `lower`. However, $\mathbf{a}\cdot\mathbf{x}$ will yield $\mathbf{b}$ only, when $\mathbf{a}$ is triangular. You should keep this in mind, when trying to establish the validity of the solution by back substitution.

In [14]:
%%micropython -unix 1

from ulab import numpy as np
from ulab import scipy as spy

a = np.array([[3, 0, 0, 0], [2, 1, 0, 0], [1, 0, 1, 0], [1, 2, 1, 8]])
b = np.array([4, 2, 4, 2])

print('a:\n')
print(a)
print('\nb: ', b)

x = spy.linalg.solve_triangular(a, b, lower=True)

print('='*20)
print('x: ', x)
print('\ndot(a, x): ', np.dot(a, x))

a:

array([[3.0, 0.0, 0.0, 0.0],
       [2.0, 1.0, 0.0, 0.0],
       [1.0, 0.0, 1.0, 0.0],
       [1.0, 2.0, 1.0, 8.0]], dtype=float64)

b:  array([4.0, 2.0, 4.0, 2.0], dtype=float64)
x:  array([1.333333333333333, -0.6666666666666665, 2.666666666666667, -0.08333333333333337], dtype=float64)

dot(a, x):  array([4.0, 2.0, 4.0, 2.0], dtype=float64)




With get the same solution, $\mathbf{x}$, with the following matrix, but the dot product of $\mathbf{a}$, and $\mathbf{x}$ is no longer $\mathbf{b}$:

In [17]:
%%micropython -unix 1

from ulab import numpy as np
from ulab import scipy as spy

a = np.array([[3, 2, 1, 0], [2, 1, 0, 1], [1, 0, 1, 4], [1, 2, 1, 8]])
b = np.array([4, 2, 4, 2])

print('a:\n')
print(a)
print('\nb: ', b)

x = spy.linalg.solve_triangular(a, b, lower=True)

print('='*20)
print('x: ', x)
print('\ndot(a, x): ', np.dot(a, x))

a:

array([[3.0, 2.0, 1.0, 0.0],
       [2.0, 1.0, 0.0, 1.0],
       [1.0, 0.0, 1.0, 4.0],
       [1.0, 2.0, 1.0, 8.0]], dtype=float64)

b:  array([4.0, 2.0, 4.0, 2.0], dtype=float64)
x:  array([1.333333333333333, -0.6666666666666665, 2.666666666666667, -0.08333333333333337], dtype=float64)

dot(a, x):  array([5.333333333333334, 1.916666666666666, 3.666666666666667, 2.0], dtype=float64)


